In [ ]:
from user_constants import *
import tpu_simulation_utilities
import tensorflow.compat.v1 as tf
import numpy as np
import os
import io
import sys
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
from skimage import transform
import ipywidgets as widgets
from osgeo import gdal
from typing import Any, Callable, Iterable, Sequence, List, Dict, Mapping, MutableMapping, MutableSequence, Optional, Text, Tuple, Union

if PUBLIC_COLAB:
  # Authenticate to access Google Cloud Storage.
  from google.colab import auth  # pylint: disable=g-import-not-at-top
  auth.authenticate_user()

def soliton_1d(y, x, L, nx, ny, dem):
    A = 15
    lamb = 2000
    #	-	Case 2
  #	A = 6
    #	-	Case 3
  #	A = 30

    # N-wave parameters
    a1 = A
    a2 = A/3
    x1 = 11000 - (0.5151125 * lamb + 2060)

    x2 = 11000 - (0.2048 * lamb + 2060)		#	Leading Depression Nwave
  #	x2 = 1000 + 0.825425 * L 	#	Leading Elevation Nwave

    k1 = 28.416 / lamb ** 2
    k2 = 256 / lamb ** 2
    z = 2 * (a1 * tf.math.exp( -k1 * (y - x1) ** 2 ) - a2 * tf.math.exp( -k2 * (y - x2) ** 2))
    z = z + 0.02*3940*(1-((y-6000)/3940))
    z = tf.where(y >9940, tf.ones_like(z)*SAINT_VENANT_EPS,z)
    z = tf.where(y<6000, tf.ones_like(z)*0.02*3940,z)
    z = tf.expand_dims(z,axis=0)
    z = tf.repeat(z, nx,axis = 0)
    return z

def run_simulation(
    dem_tiff_filename: Text, resolution: int, num_secs: float,
    num_secs_per_cycle: float, dt: float, cx: int, cy: int,
    run_dir: Text, start_time_secs: float = 0) -> None:
  """Runs the simulation."""
  # Necessary Component of Running Simulation, no editing here.
  slope = 1e-4
  input_file_format = ('{}' + f'-{start_time_secs}.np' if start_time_secs > 0
                       else None)

  unpadded_dem,geoinfo = tpu_simulation_utilities.load_dem_from_tiff_file(dem_tiff_filename,resolution)
  unpadded_river_mask = np.ones_like(unpadded_dem, dtype=bool)
  unpadded_manning_matrix = tpu_simulation_utilities.get_manning_matrix_from_river_mask(
      unpadded_river_mask, MANNING_COEFF_RIVER, MANNING_COEFF_FLOODPLAIN)
  params = tpu_simulation_utilities.get_sv_params(unpadded_dem.shape, resolution, num_secs,
                         num_secs_per_cycle, cx, cy, dt)
  print('params:', params)
  
  # SETS THE BOUNDARY CONDITIONS FOR HEIGHT, FLUX IN Y, FLUX IN X
  h_bcs = [
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.LEFT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0*3940,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.RIGHT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)
  ]

  q_y_bcs = [
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.LEFT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.DirichletBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.RIGHT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)
  ]

  q_x_bcs = [
     tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.TOP,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.BOTTOM,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)]

  # NO NEED TO EDIT THE FOLLOWING
  init_files_manager = tpu_simulation_utilities.InitFilesManager(
      params, tpu_simulation_utilities.three_d_subgrid_of_2d_grid, run_dir)

  # LAST TWO ARGUMENTS SET INITIAL CONDITIONS IN HEIGHT AND FLUX IN Y DIRECTION
  sim_builder = tpu_simulation_utilities.get_sim_builder(
      params, unpadded_dem, unpadded_manning_matrix,h_bcs,q_x_bcs,q_y_bcs,
      init_files_manager, input_file_format, start_time_secs, 
      soliton_1d, tpu_simulation_utilities.constant_height_zeros)

  def session_runner_builder(sim):
    return tpu_simulation_utilities.get_session_runner_builder(params, init_files_manager, run_dir, sim)

  manager = tpu_simulation_utilities.TPUSimulationManager(params, sim_builder, session_runner_builder)
  manager.run_simulation(start_time_secs)

In [ ]:
# SIMULATION PARAMETERS - NEED TO BE SET
resolution, dt, cx = 4, 100000, 5e-3, 1
start_time_secs, num_secs, num_secs_per_cycle = 0, 400.0,10.0
# SIMULATION DEM FILE NAME
dem_bucket_filename = f'gs://{BUCKET}/lunghino_baseline.tif'

# NO NEED TO EDIT THIS COMPONENT
dem_tiff_filename = tpu_simulation_utilities.get_dem_tiff_filename(dem_bucket_filename,resolution)
run_dir = f'gs://{BUCKET}'
if PUBLIC_COLAB:
  cy = 8
  os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [ ]:
# RUN SIMULATION ON TPU
if PUBLIC_COLAB:
  g = tf.Graph()
  run_meta = tf.RunMetadata()
  with g.as_default():
    run_simulation(dem_tiff_filename, resolution, num_secs,
                  num_secs_per_cycle, dt, cx, cy, run_dir,
                  start_time_secs)

In [ ]:
# Load the DEM and the heightmaps.
dem, geoinfo = tpu_simulation_utilities.load_dem_from_tiff_file(dem_tiff_filename,resolution)
dem_shape = dem.shape
heightmaps = {}
t_range = np.arange(start_time_secs,num_secs + num_secs_per_cycle,num_secs_per_cycle)
for t in t_range:
  t  = round(t,3)
  filename = os.path.join(run_dir, f'h-{t}.np')
  print(f'Loading heightmap: h-{t}.np')
  with tf.io.gfile.GFile(filename, 'rb') as f:
    heightmaps[t] = np.load(f)
  
resize_factor = 1
resize_shape = [dem_shape[0] // resize_factor, dem_shape[1] // resize_factor]
scaled_dem = transform.resize(dem, resize_shape)
scaled_heightmaps = {}

for ts, hmap in heightmaps.items():
  scaled_heightmaps[ts] = transform.resize(hmap, resize_shape)

In [ ]:
%matplotlib inline

def plot(time_secs):
  plt.figure(figsize=(8, 8), dpi=100)
  plt.grid(False)
  plt.title(f'Water level at time = {time_secs} seconds')
  rgb = LightSource(azdeg=315, altdeg=45).hillshade(scaled_dem[:,9500:10500], vert_exag=.005)
  plt.imshow(rgb, alpha=1, cmap='gray', vmin=-.5, vmax=.9)
  x=scaled_heightmaps[time_secs][:,9500:10500]
  norm_x = .5 + .5 * (x / 10)

  y = plt.cm.Blues(norm_x)
  y[..., -1] = x / 2
  plt.imshow(y)
  plt.show()

if PUBLIC_COLAB:
  slider = widgets.FloatSlider(
      value=num_secs,
      min=start_time_secs, max=num_secs,
      step=num_secs_per_cycle)
  out = widgets.interactive_output(
      lambda time_secs: plot(time_secs), {'time_secs': slider})
  display(widgets.VBox([slider, out]))
